In [1]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [2]:
self = MolecularDynamics(number_atoms=[50, 2],
                      epsilon=[0.1, 0.1], # kcal/mol
                      sigma=[2, 1], # A
                      atom_mass= [1, 1], # g/mol
                      Lx=30, # A
                      Ly=30, # A
                      Lz=30, # A
                      minimization_steps = 10,
                      maximum_steps=1000,
                      desired_temperature=300,
                      thermo = 10,
                      dump = 10,  
                      tau_temp = 100, # fs
                      #tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps")

step epot maxF
0 -0.586 18.431
5 -0.716 1.734
10 -0.909 2.057
 step     N  temp  epot  ekin press   vol
    0    52 299.96 -0.9020334286363186 45.59967745093045 75.31 27000.0
   10    52 299.29 -0.7951387135459331 45.4988880117466  75.4 27000.0
   20    52 298.58 -0.669210226073088 45.39022854595751 76.06 27000.0
   30    52 298.41 -0.6188475105952014 45.363872260049845 75.67 27000.0
   40    52 298.47 -0.6042648444941618 45.373322949238506 75.01 27000.0
   50    52 298.63 -0.6071540288189617 45.39786863886687 75.32 27000.0
   60    52 295.26 -0.05434859645301472 44.88639192065615 82.61 27000.0
   70    52 296.28 -0.13905348496197723 45.04024342587797 82.12 27000.0
   80    52 297.19 -0.2187852550424165 45.17937725160431 80.16 27000.0
   90    52 299.64 -0.5721839821299619 45.55095067236016 75.78 27000.0
  100    52 295.41 0.07563358157911898 44.9088883374277 83.73 27000.0
  110    52 298.44 -0.23788986569410603 45.36911886696015 80.61 27000.0
  120    52 300.02 -0.472102114607514 45.6

0